# Data Engineering

## Question 1. Understanding docker first run
Run docker with the python:3.13 image in an interactive mode, use the entrypoint bash.

What's the version of pip in the image?
- 25.3

nevin@np:~/GitHub/de-zoomcamp$ docker run -it --entrypoint bash python:3.13
Unable to find image 'python:3.13' locally
3.13: Pulling from library/python
2ca1bfae7ba8: Pull complete 
82e18c5e1c15: Pull complete 
be442a7e0d6f: Pull complete 
26d823e3848f: Pull complete 
ca4b54413202: Pull complete 
b6513238a015: Pull complete 
9b57076d00d4: Pull complete 
Digest: sha256:c8b03b4e98b39cfb180a5ea13ae5ee39039a8f75ccf52fe6d5c216eed6e1be1d
Status: Downloaded newer image for python:3.13
root@db4f8d429a45:/# pip --version
pip 25.3 from /usr/local/lib/python3.13/site-packages/pip (python 3.13)
root@db4f8d429a45:/# 

## Question 2. Understanding Docker networking and docker-compose
what is the hostname and port that pgadmin should use to connect to the postgres database?
- db:5432

 The db service exposes port 5432 internally (mapped to 5433 on the host). Therefore, pgAdmin should connect to db on port 5432.

## Prepare Postgres

In [ ]:
!wget -O data/green_tripdata_2025-11.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet


--2026-01-27 21:10:47--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2684:7600:b:20a5:b140:21, 2600:9000:2684:6a00:b:20a5:b140:21, 2600:9000:2684:1600:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:2684:7600:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1164775 (1.1M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2025-11.parquet’

data/green_tripdata 100%[===================>]   1.11M  2.94MB/s    in 0.4s    

2026-01-27 21:10:48 (2.94 MB/s) - ‘data/green_tripdata_2025-11.parquet’ saved [1164775/1164775]



In [7]:
!wget -O data/taxi_zone_lookup.csv https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

--2026-01-27 21:12:19--  https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv
Resolving github.com (github.com)... 20.26.156.215
Connecting to github.com (github.com)|20.26.156.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/513814948/5a2cc2f5-b4cd-4584-9c62-a6ea97ed0e6a?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-27T22%3A11%3A33Z&rscd=attachment%3B+filename%3Dtaxi_zone_lookup.csv&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-01-27T21%3A11%3A12Z&ske=2026-01-27T22%3A11%3A33Z&sks=b&skv=2018-11-09&sig=hcflCdPXespAS%2F0WghmR4P0NmBjyxNIlN6z%2FfVvt8wM%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2OTU0ODY0MCwibmJmIjoxNzY5NTQ4MzQwLCJwYXRoIjoicmVsZWFzZWFzc

## Question 3. Counting short trips
For the trips in November 2025 (lpep_pickup_datetime between '2025-11-01' and '2025-12-01', exclusive of the upper bound), how many trips had a trip_distance of less than or equal to 1 mile?


- 8,007


-------question 3: Counting short trips

ny_taxi=# SELECT COUNT(*) AS short_trips
FROM green_tripdata
WHERE lpep_pickup_datetime >= '2025-11-01'
  AND lpep_pickup_datetime < '2025-12-01'
  AND trip_distance <= 1;
 short_trips 
-------------
        8007
(1 row)

## Question 4. Longest trip for each day
Which was the pick up day with the longest trip distance? Only consider trips with trip_distance less than 100 miles (to exclude data errors).

Use the pick up time for your calculations.

- 2025-11-14



-------question 4: longest trip for each day
ny_taxi=# SELECT 
    DATE(lpep_pickup_datetime) AS pickup_date,
    MAX(trip_distance) AS max_trip_distance
FROM green_tripdata
WHERE trip_distance < 100
GROUP BY DATE(lpep_pickup_datetime)
ORDER BY max_trip_distance DESC
LIMIT 1;
 pickup_date | max_trip_distance 
-------------+-------------------
 2025-11-14  |             88.03
(1 row)

## Question 5. Biggest pickup zone
Which was the pickup zone with the largest total_amount (sum of all trips) on November 18th, 2025?

- East Harlem North





--------question5: Biggest pickup zones 
ny_taxi=# SELECT 
    z."Zone" AS pickup_zone,
    SUM(g.total_amount) AS total_revenue
FROM green_tripdata g
JOIN taxi_zone_lookup z
  ON g."PULocationID" = z."LocationID"
WHERE DATE(g.lpep_pickup_datetime) = '2025-11-18'
GROUP BY z."Zone"
ORDER BY total_revenue DESC
LIMIT 1;
    pickup_zone    |   total_revenue   
-------------------+-------------------
 East Harlem North | 9281.919999999991
(1 row)

## Question 6. Largest tip
For the passengers picked up in the zone named "East Harlem North" in November 2025, which was the drop off zone that had the largest tip?

Note: it's tip , not trip. We need the name of the zone, not the ID.


- Yorkville West



----question 6:largest tip
ny_taxi=# SELECT dz."Zone" AS dropoff_zone, g.tip_amount
FROM green_tripdata g
JOIN taxi_zone_lookup pz
  ON g."PULocationID" = pz."LocationID"
JOIN taxi_zone_lookup dz
  ON g."DOLocationID" = dz."LocationID"
WHERE pz."Zone" = 'East Harlem North'
  AND g.lpep_pickup_datetime >= '2025-11-01'
  AND g.lpep_pickup_datetime < '2025-12-01'
ORDER BY g.tip_amount DESC
LIMIT 1;
  dropoff_zone  | tip_amount 
----------------+------------
 Yorkville West |      81.89
(1 row)

## Question 7. Terraform Workflow
Which of the following sequences, respectively, describes the workflow for:

- Downloading the provider plugins and setting up backend,
- Generating proposed changes and auto-executing the plan
- Remove all resources managed by terraform`
Answers:


- terraform init, terraform apply -auto-approve, terraform destroy
